# 🎯 Cross-Camera Player Mapping
This notebook detects and matches players across two camera views using YOLOv11 and the Hungarian algorithm.

In [2]:
from google.colab import files

# Upload your model and videos
uploaded = files.upload()

Saving broadcast.mp4 to broadcast (1).mp4


In [3]:
# ✅ Install required libraries (run this once)
%pip install ultralytics opencv-python-headless scipy numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [5]:
from ultralytics import YOLO

# Load the fine-tuned YOLOv11 model (adjust filename if needed)
model = YOLO('yolov8n.pt')


100%|██████████| 6.25M/6.25M [00:00<00:00, 121MB/s]


In [6]:
import cv2

def detect_players(video_path, model):
    cap = cv2.VideoCapture(video_path)
    detections = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        detections.append(results[0])
    cap.release()
    return detections


In [7]:
def extract_features(detections):
    features = []
    for det in detections:
        frame_features = []
        if det.boxes is not None:
            for box in det.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                center = ((x1 + x2) / 2, (y1 + y2) / 2)
                frame_features.append({'bbox': (x1, y1, x2, y2), 'center': center})
        features.append(frame_features)
    return features


In [8]:
import numpy as np

def average_frame_features(features):
    avg_positions = {}
    counts = {}
    for frame in features:
        for i, player in enumerate(frame):
            if i not in avg_positions:
                avg_positions[i] = np.array(player['center'])
                counts[i] = 1
            else:
                avg_positions[i] += np.array(player['center'])
                counts[i] += 1
    for i in avg_positions:
        avg_positions[i] /= counts[i]
    return avg_positions


In [9]:
from scipy.optimize import linear_sum_assignment

def match_players(broadcast_avg, tacticam_avg):
    b_ids = list(broadcast_avg.keys())
    t_ids = list(tacticam_avg.keys())
    cost_matrix = np.zeros((len(b_ids), len(t_ids)))
    for i, b in enumerate(b_ids):
        for j, t in enumerate(t_ids):
            cost_matrix[i, j] = np.linalg.norm(broadcast_avg[b] - tacticam_avg[t])
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return {t_ids[col]: b_ids[row] for row, col in zip(row_ind, col_ind)}


In [10]:
# 🔁 Run the full pipeline
broadcast_detections = detect_players('broadcast.mp4', model)
tacticam_detections = detect_players('tacticam.mp4', model)

broadcast_features = extract_features(broadcast_detections)
tacticam_features = extract_features(tacticam_detections)

broadcast_avg = average_frame_features(broadcast_features)
tacticam_avg = average_frame_features(tacticam_features)

player_mapping = match_players(broadcast_avg, tacticam_avg)
print("✅ Player Mapping:", player_mapping)



0: 384x640 2 persons, 1 sports ball, 49.2ms
Speed: 16.6ms preprocess, 49.2ms inference, 428.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 sports ball, 8.0ms
Speed: 4.5ms preprocess, 8.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 7.4ms
Speed: 4.4ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 1 tennis racket, 7.1ms
Speed: 4.1ms preprocess, 7.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 sports ball, 8.6ms
Speed: 4.3ms preprocess, 8.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 8.2ms
Speed: 3.6ms preprocess, 8.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 6.8ms
Speed: 4.3ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.

In [11]:
import json

with open('player_id_mapping.json', 'w') as f:
    json.dump(player_mapping, f, indent=4)

print("📝 Saved player mapping to 'player_id_mapping.json'")


📝 Saved player mapping to 'player_id_mapping.json'
